In [1]:
import pandas as pd
import numpy as np
import datetime
import re

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

from google.colab import drive
drive.mount('/content/drive')


Authenticated
Mounted at /content/drive


In [109]:
%%bigquery --project gcp-wow-ent-im-tbl-prod article_pos_summary

#standardSQL
with dat as (
    SELECT 
distinct
 
a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom, 
PERCENTILE_CONT(a.retailquantity, 0.5) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS median_retailquantity,
PERCENTILE_CONT(a.retailquantity, 0.1) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS quantile_10th_retailquantity,
PERCENTILE_CONT(a.retailquantity, 0.9) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS quantile_90th_retailquantity,
PERCENTILE_CONT(a.retailquantity, 0.01) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS quantile_1st_retailquantity,
PERCENTILE_CONT(a.retailquantity, 0.99) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS quantile_99th_retailquantity,

-- PERCENTILE_CONT(a.retailamount, 0.5) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS median_retailamount,
-- PERCENTILE_CONT(a.retailamount, 0.1) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS quantile_10th_retailamount,
-- PERCENTILE_CONT(a.retailamount, 0.9) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS quantile_90th_retailamount,

PERCENTILE_CONT((case when a.retailquantity = 0 then null else a.retailamount/a.retailquantity end), 0.5) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS median_retailamount_per_retailquantity,
PERCENTILE_CONT((case when a.retailquantity = 0 then null else a.retailamount/a.retailquantity end), 0.1) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS quantile_10th_retailamount_per_retailquantity,
PERCENTILE_CONT((case when a.retailquantity = 0 then null else a.retailamount/a.retailquantity end), 0.9) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS quantile_90th_retailamount_per_retailquantity,
PERCENTILE_CONT((case when a.retailquantity = 0 then null else a.retailamount/a.retailquantity end), 0.01) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS quantile_1st_retailamount_per_retailquantity,
PERCENTILE_CONT((case when a.retailquantity = 0 then null else a.retailamount/a.retailquantity end), 0.99) over(partition by a.article, c.ArticleDescription, b.Sub_CategoryDescription, a.retailuom) AS quantile_99th_retailamount_per_retailquantity,

FROM
`gcp-wow-ent-im-tbl-prod.adp_dm_basket_sales_view.pos_item_line_detail` a,
`gcp-wow-ent-im-tbl-prod.adp_dm_masterdata_view.dim_article_hierarchy_v` b,
`gcp-wow-ent-im-tbl-prod.adp_dm_masterdata_view.dim_article_v` c
where
(ltrim(c.Article,'0')=ltrim(b.Article,'0')) and
a.salesorg = '1005'
and ltrim(a.article,'0') = ltrim(b.article,'0')
and b.salesorg = '1005'
and b.DepartmentDescription like '%MEAT%'
--and b.SegmentDescription = 'LAMB'
and b. Department not in ('W100','W120')
and businessdate >= date_sub(current_date(), interval 90*1 day) 
and itemvoidflag is null
and ItemTXNType in  ('S201') #,'S202') --S201 sales, S202 returns.
order by a.article, a.retailuom
),
split_ArticleDescription as (
select split( REGEXP_REPLACE(lower(ArticleDescription), r'[\s]{2,}', ' '),' ') as split_ArticleDescription, *
from dat
)
--select retailuom, Sub_CategoryDescription, article, ArticleDescription, flattened_split_ArticleDescription, quantile_90th_retailamount_per_retailquantity
--from split_ArticleDescription, split_ArticleDescription.split_ArticleDescription as flattened_split_ArticleDescription
--select retailuom, Sub_CategoryDescription, article, REGEXP_REPLACE(lower(ArticleDescription), r'[\s]{2,}', ' ') as ArticleDescription, quantile_90th_retailquantity, quantile_90th_retailamount_per_retailquantity
select retailuom, Sub_CategoryDescription, article, REGEXP_REPLACE(lower(ArticleDescription), r'[\s]{2,}', ' ') as ArticleDescription, quantile_99th_retailquantity, quantile_99th_retailamount_per_retailquantity
from dat
where retailuom = 'KG'

In [110]:
#Get the 'primal mapping gsheet'
#=========================================================================================================================================================================================================================
!pip install gspread --upgrade # Uninstall gspread-3.0.1 with 3.7.0
!pip install gspread-dataframe --upgrade
import gspread
from oauth2client.client import GoogleCredentials as GC
GoogleSheets = gspread.authorize(GC.get_application_default())
import gspread_dataframe 

wb = GoogleSheets.open_by_url('https://docs.google.com/spreadsheets/d/16Oam02bvZOEedDDajJ255fNZaZLSj7b27ewQEQFuufY/edit#gid=1704440079')


In [111]:
def function_gsheet_data(wb=wb, wsName='Sheet1'):
  mWB = wb.worksheet(wsName)
  dat = mWB.get_all_values()
  df = pd.DataFrame(dat, columns=['Supplier',	'PRODUCTION_LINE',	'Type',	'PRIMAL_CODE',	'Primal',	'WOW_Code', 'Description',	'CRATE_WEIGHT',	'YIELD'])
  df = df.iloc[1:] # removes column header row from 1st data row
  return df

In [112]:
primal_mapping = function_gsheet_data()

In [113]:
primal_mapping.head()

,Supplier,PRODUCTION_LINE,Type,PRIMAL_CODE,Primal,WOW_Code,Description,CRATE_WEIGHT,YIELD
1,88139003,BS,Butcher Shop,(blank),(blank),944864,LAMB SADDLE,4,1.00
2,88139003,BS,Butcher Shop,108604,MSA Rump Cap Butcher Shop,108604,MSA RUMP CAP 5 DAYS FOR BUTCHER IW/VAC,15,1.00
3,88139003,BS,Butcher Shop,160138,MSA Beef Ribs Prepared,160138,MSA BEEF RIBS PREPARED,15,1.00
4,88139003,BS,Butcher Shop,160147,Msa Beef Rump Ctn,160147,MSA BEEF RUMP CTN,14.52,1.00
5,88139003,BS,Butcher Shop,160148,Msa Beef Shortloin Ctn,160148,MSA BEEF SHORTLOIN CTN,12.66,1.00


In [114]:
# clean string
def string_clean(s):
  s = s.str.lower()
  s = [re.sub('\s{2,}', ' ', x) for x in s]
  s = [re.sub('[^a-zA-Z0-9 ]', ' ', x) for x in s]
  s = [x.strip() for x in s]
  return s

primal_mapping['Description'] = string_clean(primal_mapping['Description'])
primal_mapping['Primal'] = string_clean(primal_mapping['Primal'])
primal_mapping['Type'] = string_clean(primal_mapping['Type'])

In [115]:
primal_mapping.head()

,Supplier,PRODUCTION_LINE,Type,PRIMAL_CODE,Primal,WOW_Code,Description,CRATE_WEIGHT,YIELD
1,88139003,BS,butcher shop,(blank),blank,944864,lamb saddle,4,1.00
2,88139003,BS,butcher shop,108604,msa rump cap butcher shop,108604,msa rump cap 5 days for butcher iw vac,15,1.00
3,88139003,BS,butcher shop,160138,msa beef ribs prepared,160138,msa beef ribs prepared,15,1.00
4,88139003,BS,butcher shop,160147,msa beef rump ctn,160147,msa beef rump ctn,14.52,1.00
5,88139003,BS,butcher shop,160148,msa beef shortloin ctn,160148,msa beef shortloin ctn,12.66,1.00


In [116]:
article_pos_summary.head()

,retailuom,Sub_CategoryDescription,article,ArticleDescription,quantile_99th_retailquantity,quantile_99th_retailamount_per_retailquantity
0,KG,PORK,100375,pork bones kg 2,5.40400,3.989637
1,KG,SERVICED MEAT,101196,lamb shank,1.90766,14.548782
2,KG,VEAL,108045,stir fry veal heart smart kg 3,0.25000,17.000000
3,KG,SERVICED MEAT,109363,beef bones,2.01018,6.017152
4,KG,SERVICED MEAT,110398,beef picanha,3.66780,22.004002


In [117]:
primal_mapping.reset_index(drop=True).to_csv('primal_mapping.csv')
article_pos_summary.reset_index(drop=True).to_csv('article_pos_summary.csv')

In [118]:
# activate R magic
#!pip install rpy2==3.2.2
!pip install rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [119]:
%%R #-i all_change_request_data

library(tidyverse)
#if (!require("feather")) install.packages("feather")
install.packages("feather")
library(feather)

install.packages("stringdist")
library(stringdist)

install.packages("reshape2")
library(reshape2)

df <- read_csv("article_pos_summary.csv")
arts <- df %>%
  filter(retailuom=='KG') %>%
  distinct(ArticleDescription) %>%
  .[[1]]

arts

#### primal_mapping
primalMap_df <- read_csv("primal_mapping.csv")

primals <- primalMap_df %>%
  distinct(Description) %>%
  .[[1]]

primals


#x <- stringdistmatrix(arts,arts,method="qgram" ,q=2)
x <- stringdistmatrix(primals,arts,method="cosine" ,q=2)
rownames(x) <- primals
colnames(x) <- arts
x

saveRDS(x, "Primals_articles_stringdist.rds")
write.csv(x, "Primals_articles_stringdist.csv")

long_x <- reshape2::melt(x) %>% data.frame() %>% rename("Description"="Var1", "Article_Description"="Var2", "string_distance"="value") #%>% filter(string_distance < 0.2) 
saveRDS(long_x, "long_x.rds")
write.csv(long_x, "long_Primals_articles_stringdist.csv", row.names = FALSE, quote = FALSE)


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/feather_0.3.5.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 73812 bytes (72 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write 

Rows: 507 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): retailuom, Sub_CategoryDescription, ArticleDescription
dbl (4): ...1, article, quantile_99th_retailquantity, quantile_99th_retailam...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: New names:
* `` -> ...1



Rows: 1113 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): PRODUCTION_LINE, Type, PRIMAL_CODE, Primal, Description
dbl (5): ...1, Supplier, WOW_Code, CRATE_WEIGHT, YIELD

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [120]:
long_Primals_articles_stringdist = pd.read_csv("long_Primals_articles_stringdist.csv", dtype={'Primal':'string','Primal':'string','string_distance':'float64'})

In [121]:
long_Primals_articles_stringdist.head()

,Description,Article_Description,string_distance
0,lamb saddle,pork bones kg 2,1.000000
1,msa rump cap 5 days for butcher iw vac,pork bones kg 2,0.874782
2,msa beef ribs prepared,pork bones kg 2,0.888544
3,msa beef rump ctn,pork bones kg 2,0.933185
4,msa beef shortloin ctn,pork bones kg 2,0.883358


In [122]:
addit_long_Primals_articles_stringdist = long_Primals_articles_stringdist.merge(article_pos_summary,left_on='Article_Description',right_on='ArticleDescription')

In [123]:
print(addit_long_Primals_articles_stringdist.shape)
print(long_Primals_articles_stringdist.shape)

(244881, 9)
(205275, 3)


In [124]:
#primal_mapping.merge(long_Primals_articles_stringdist, left_on='Description', right_on='Description')
primal_mapping_summary = primal_mapping.merge(addit_long_Primals_articles_stringdist, left_on='Description', right_on='Description', how='left')

In [125]:
primal_mapping_summary.head()

,Supplier,PRODUCTION_LINE,Type,PRIMAL_CODE,Primal,WOW_Code,Description,CRATE_WEIGHT,YIELD,Article_Description,string_distance,retailuom,Sub_CategoryDescription,article,ArticleDescription,quantile_99th_retailquantity,quantile_99th_retailamount_per_retailquantity
0,88139003,BS,butcher shop,(blank),blank,944864,lamb saddle,4,1.00,pork bones kg 2,1.000000,KG,PORK,100375,pork bones kg 2,5.40400,3.989637
1,88139003,BS,butcher shop,(blank),blank,944864,lamb saddle,4,1.00,lamb shank,0.472954,KG,SERVICED MEAT,101196,lamb shank,1.90766,14.548782
2,88139003,BS,butcher shop,(blank),blank,944864,lamb saddle,4,1.00,stir fry veal heart smart kg 3,0.948012,KG,VEAL,108045,stir fry veal heart smart kg 3,0.25000,17.000000
3,88139003,BS,butcher shop,(blank),blank,944864,lamb saddle,4,1.00,beef bones,1.000000,KG,SERVICED MEAT,109363,beef bones,2.01018,6.017152
4,88139003,BS,butcher shop,(blank),blank,944864,lamb saddle,4,1.00,beef bones,1.000000,KG,SERVICED MEAT,175263,beef bones,1.53120,6.006891


In [126]:
primal_mapping_summary.tail()['Primal'].iloc[0]

'msa ribs prepared   4 rib iw vac crate'

In [127]:
print(primal_mapping.shape)
print(primal_mapping_summary.shape)

(1113, 9)
(564291, 17)


In [128]:
long_Primals_articles_stringdist.query("string_distance>0").sort_values('string_distance')

,Description,Article_Description,string_distance
23969,free range pork cutlet,free range pork cutlets,0.021055
32669,ww thin chkn fetta spinach sausages,ww thin chkn fetta & spinach sausages,0.063618
40128,beef 100 day grain fed bolar blade,100 day grain fed beef bolar blade who,0.068728
178308,grass fed beef eye fillet,msa grass fed beef eye fillet,0.069051
154398,msa grass fed beef eye fillet,grass fed beef eye fillet,0.069051
...,...,...,...
112756,amelia park lamb shldr bfly suvlki rw,beef eye fillet whole,1.000000
112752,lamb hearts,beef eye fillet whole,1.000000
112751,lamb liver,beef eye fillet whole,1.000000
112268,lamb liver,beef eye fillet steak,1.000000


## Save to bq

In [129]:
bq_primal_mapping_summary = primal_mapping_summary[['Primal','WOW_Code','PRIMAL_CODE','Article_Description','Sub_CategoryDescription','retailuom','string_distance','quantile_99th_retailquantity','quantile_99th_retailamount_per_retailquantity']].drop_duplicates()

In [130]:
pd.io.gbq.to_gbq(bq_primal_mapping_summary, 'sandpit.primal_mapping_summary', 'gcp-wow-pvc-grnstck-prod', chunksize=100000, reauth=False, if_exists='replace')

3it [00:22,  7.55s/it]
